In [7]:
%pip install pymongo
%pip install heapq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement heapq (from versions: none)
ERROR: No matching distribution found for heapq


In [8]:
import json
from pymongo import MongoClient
from heapq import heapify, heappush, heappop
import pymongo
import time
from mysql.connector import (connection)
import mysql.connector

In [10]:
class ProjectCache:
    
    def __init__(self, database):
        self.cache = {}                                      #Cache
        self.minheap = []                                    #Minheap
        self.cache_limit_value = 200                         #The cache will be set up to only hold 200 tweets
        self.db = MongoClient(database)["my_final_database"]
        self.tweets = self.db["tweets"]
        ktdm = list(self.db["keywords"].find())              
        htdm = list(self.db["hashtags"].find())
        self.keyword_tdm = dict((x['_id'], x['tweets']) for x in ktdm)    #Holds all keywords and ids associated
        self.hashtag_tdm = dict((x['_id'], x['tweets']) for x in htdm)    #Holds all hashtags and ids associated
        self.total_searches_kept_in_cache = 10      #Each search cache can hold up to 10 most recent unique searches
        self.recent_keyword_searches_cache = {}                      #Text search cache1
        self.recent_hashtag_searches_cache = {}                      #Hashtag search cache1
        self.recent_keyword_searches = []
        self.recent_hashtag_searches = []
        self.connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
        self.users_keyword_cache = {}                                #Text search cache2
        self.users_keyword_list = []
        self.users_hashtag_cache = {}                                #Hashtag search cache2
        self.users_hashtag_list = []
        self.recent_user_searches_cache= {}                          #User search cache1
        self.recent_user_searches = []
        self.tweets_user_cache = {}                                  #User search cache2
        self.tweets_user_list = []
        
    def __contains__(self, key):
        
        return key in self.cache
    
    def current_cache_size(self):
        
        return len(self.cache)
    
    def initial_population(self):
        
        #If the cache database already exists in mongoDB database then export it
        if self.db["cache"] != None:
            self.cache = self.db["cache"]
            return
        
        #Read every tweet in collection and attempt to add them one by one to cache
        for i in self.tweets:
            add_update(i["id"], i)
           
    def add_update(self, key, value):
        
        #Add a section that first checks if the current key's relevance value is greater 
        #than the least relevant term in the cache
        if key not in self.cache:
            #If the cache limit is met get the least relevant in cache then compare current item with least relevant in cache
            if len(self.cache) >= self.cache_limit_value:  
                least_relevant_term()                                       
                if value["relevance"] > least_relevant_value_in_cache:      
                    
                    #then remove least relevant to make room in cache
                    self.remove_least_relevant(least_relevant_key, least_relevant_value_in_cache)          
                    
                    #Add key and relevance value to minheap list
                    minheap_sort(key, value["relevance"])
                    
                    #Then add key and all tweet elements to cache
                    self.cache[key] = value                  
        
            else:
                #Add key and relevance value to minheap list
                minheap_sort(key, value["relevance"])
                
                #Then add key and all tweet elements to cache
                self.cache[key] = value
        
    def remove_least_relevant(self, least_relevant_key, least_relevant_value_in_cache):
        
        #Remove least relevant from both the cache and the heaped list
        self.cache.pop(least_relevant)
        heappop(self.minheap, (least_relevant_value_in_cache, least_relevant))
        heapify(self.minheap)
        
    def least_relevant_term(self):
        
        #Get the least relevant term within the cache as well as key associated with it.      
        least_relevant_value_in_cache = self.minheap[0][0]
        least_relevant_key = self.minheap[0][1]
        return least_relevant_value_in_cache, least_relevant_key
    
    def minheap_sort(self, key, value):
        
        #The relevance will be the new key, and the key will be the value in the minheap.      
        heappush(self.minheap, (value, key))
        heapify(self.minheap)
    
    
    def keyword(self, word, querytype):
        
        #Make a blank dictionary
        l={}
        
        #Grab all the tweet ids associated with the word/hashtag
        if querytype == "Text":
            #If word is in searched cache just return that dictionary
            if word in self.recent_keyword_searches_cache:
                self.recent_keyword_searches.remove(word)
                self.recent_keyword_searches.append(word)
                l = self.recent_keyword_searches_cache[word]
                return pd.DataFrame.from_dict(l).transpose()
            else:
                ids = self.keyword_tdm[word] #Could use .head(10) to get only the top 10
        elif querytype == "Hashtag":
            #If word is in searched cache just return that dictionary
            if word in self.recent_hashtag_searches_cache:
                self.recent_hashtag_searches.remove(word)
                self.recent_hashtag_searches.append(word)
                l = self.recent_hashtag_searches_cache[word]
                return pd.DataFrame.from_dict(l).transpose()
            else:
                ids = self.hashtag_tdm[word]
        
        #Search through cache first for all tweet ids
        #If it's not in the cache find it in the database
        for i in ids:
            if i in self.cache:
                l[i] = self.cache[i]
            else:
                l[i] = self.connection_name.find({"id":i})
        
        #make a nested dictionary with the three most recent searched words saved in a queue
        if querytype == "Text":
            self.recent_keyword_searches_cache[word] = l
            self.recent_keyword_searches.append(word)
            #If the queue is larger than the total accepted searches remove 
            if len(self.recent_keyword_searches_cache) > self.total_searches_kept_in_cache:
                a = self.recent_keyword_searches.pop(0)
                self.recent_keyword_searches_cache.pop(a)
        elif querytype == "Hashtag":
            self.recent_hashtag_searches_cache[word] = l
            self.recent_hashtag_searches.append(word)
            #If the queue is larger than the total accepted searches remove 
            if len(self.recent_hashtag_searches_cache) > self.total_searches_kept_in_cache:
                a = self.recent_hashtag_searches.pop(0)
                self.recent_hashtag_searches_cache.pop(a)
            
        l = pd.DataFrame.from_dict(l).transpose()
        
        #Return a dictionary of all of the ids and associated tweets
        return l, word, querytype
        
    def userlist(self, l, word, querytype):
        
        #returns users already in text or hashtag cache
        if querytype == "Text":
            if word in self.users_keyword_cache:
                userCounts = self.users_keyword_cache[word]
                return userCounts.head(5)
            
        elif querytype == "Hashtag":
            if word in self.users_keyword_cache:
                userCounts = self.users_keyword_cache[word]
                return userCounts.head(5)
            
        userCounts = l.user.value_counts().to_frame(name = 'relevant_tweets')
        userCounts['user_id'] = list(userCounts.index)
     
        #Query mySQL for users not found in cache
        query = """
        SELECT *
        FROM users_table 
        WHERE id IN (%s)
        """ % ', '.join(str(x) for x in userCounts.user_id)
              
        userList = pd.read_sql(query, connection1)
        
        #Merge user_id of tweets with id of users
        userCounts = userCounts.merge(userList, left_on = 'user_id', right_on = 'id')
        
        userCounts = userCounts.sort_values(by=['relevant_tweets', 'followers_count', 'friends_count'], ascending = False)
        
        #Add word used in search to cache for users
        if querytype == "Text":    
            self.users_keyword_cache[word] = userCounts
            self.users_keyword_list.append(word)
            #If the queue is larger than the total accepted searches remove 
            if len(self.users_keyword_list) > self.total_searches_kept_in_cache:
                a = self.users_keyword_list.pop(0)
                self.users_keyword_cache.pop(a)
                
        elif querytype == "Hashtag":
            self.users_hashtag_cache[word] = userCounts
            self.users_hashtag_list.append(word)
            #If the queue is larger than the total accepted searches remove 
            if len(self.users_hashtag_list) > self.total_searches_kept_in_cache:
                a = self.users_hashtag_list.pop(0)
                self.users_hashtag_cache.pop(a)
                
        #Returns the five most active users
        return userCounts.head(5)
    
    def user(self, username):
        #Make a blank dictionary
        l={}
        
        #Grab all the tweet ids associated with the word/hashtag
        #If word is in searched cache just return that dictionary
        if username in self.recent_user_searches_cache:
            self.recent_user_searches.remove(username)
            self.recent_user_searches.append(username)
            l = self.recent_user_searches_cache[username]
            return pd.DataFrame.from_dict(l).transpose()
        else:
            query = """
            SELECT *
            FROM new_database.users_table 
            WHERE name LIKE '%{}%' OR screen_name LIKE '%{}%'
            """.format(username, username)
            #Query mySQL for users not found in cache
            userList = pd.read_sql(query, self.connection1)
            
            userList = userList.sort_values(by=['followers_count', 'friends_count'], ascending = False)
            
            self.recent_user_searches_cache[username] = userList
            self.recent_user_searches.append(username)
            #If the queue is larger than the total accepted searches remove 
            if len(self.recent_user_searches_cache) > self.total_searches_kept_in_cache:
                a = self.recent_user_searches.pop(0)
                self.recent_user_searches_cache.pop(a)
            return userList.head(5)
       
        
    def tweetlist(self, userlist, username):
        
        #If username in recent search list return tweets associated to user
        if username in self.tweets_user_cache:
            self.tweets_user_list.remove(username)
            self.tweets_user_list.append(username)
            l = self.tweets_user_cache[username]
            return pd.DataFrame.from_dict(l).transpose()
        
        tweet_ids = [x for sl in [x.split(',') for x in userlist['tweets']] for x in sl]
        #Make a blank dictionary
        l={}
        
        #Search through cache first for all tweet ids
        #If it's not in the cache find it in the database
        for i in tweet_ids:
            if i in self.cache:
                l[i] = self.cache[i]
            else:
                l[i] = self.tweets.find_one({"_id":int(i)})
        
        self.tweets_user_cache[username] = l
        self.tweets_user_list.append(username)
        #If the queue is larger than the total accepted searches remove 
        if len(self.tweets_user_cache) > self.total_searches_kept_in_cache:
            a = self.tweets_user_list.pop(0)
            self.tweets_user_cache.pop(a)
          
        return pd.DataFrame.from_dict(l).transpose()
    
    def write_to_disc(self):
        
        #inserts cache as a dictionary into cache database
        self.db["cache"].insert(self.cache)